In [2]:
import pandas as pd
import re

In [114]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


read data, resume_data.pkl is the output from data_cleaning.ipynb where skills have already been identified for each resume

In [4]:
resume_data = pd.read_pickle("resume_data.pkl")

In [6]:
annotations = resume_data["annotation"].values

In [14]:
labels = []
for annotation in annotations:
    for point in annotation:
        if len(point["label"])>0:
            if point["label"][0] not in labels:
                labels.append(point["label"][0])

In [15]:
labels

['Skills',
 'College Name',
 'Graduation Year',
 'Designation',
 'Companies worked at',
 'Email Address',
 'Location',
 'Name',
 'Degree',
 'Years of Experience',
 'UNKNOWN']

## add degree ans designation along with skills to feature set

In [108]:
feature_set = ['Designation', "Degree", "Skills"]
stopwords = ["in", "are", "is", "it", "of", "less", "more", "than", "approx", "the", "and", "to", "with", "year", "years", "under", "yrs", "additional", "skill", "skills"]

In [122]:
def clean_text(txt):
    "Trim and clean the text of special characters"
    txt = txt.lower()
    txt = re.sub("[^a-zA-Z]+", " ", txt)
    txt = re.sub("\s+", " ", txt)
    txt = txt.strip()
    txt = [lemmatizer.lemmatize(y.strip()) for y in txt.split() if y.strip() not in stopwords]
    return " ".join(txt)


In [121]:
def process_all_label(annotation, label):
    """identify points with skill label and process each set"""
    label_annots = [x for x in annotation if label in x['label']]
    label_set = []
    for label_point in label_annots:
        for point in label_point["points"]:
            label_text = point["text"]
            temp = clean_text(label_text)
            label_set.append(temp)
    return list(set(label_set))


In [110]:
def desig_degree(annotation):
    """identify degree and designation"""
    degree = process_all_label(annotation, "Degree")
    desig = process_all_label(annotation, "Designation")
    return list(set(degree+desig))

In [123]:
desig_degree(resume_data["annotation"][2])

['electrical electronics engineering', 'senior system engineer']

In [124]:
resume_data["desig_degree"] = resume_data["annotation"].apply(desig_degree)
resume_data["feature_set"] = resume_data["skills"].apply(lambda x: [clean_text(y) for y in x])+resume_data["desig_degree"]

In [125]:
resume_data.to_pickle("resume_data.pkl")